In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 2.8.0+cu126 _CudaDeviceProperties(name='NVIDIA A100-SXM4-80GB', major=8, minor=0, total_memory=81221MB, multi_processor_count=108, uuid=ad05a8ee-fedd-7862-940b-41b0a52f4b69, pci_bus_id=0, pci_device_id=5, pci_domain_id=0, L2_cache_size=40MB)


In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button");
     btn.click()
     }

   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect");
     btn.click()
     }
  }

setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [ ]:
%cd /content/drive/MyDrive/colab/detect/yolov5
!pip install -U -r requirements.txt

[Errno 2] No such file or directory: '/content/drive/MyDrive/colab/detect/yolov5'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
%cd /content/drive/MyDrive/黃俊翔_論文研究📚/Street_view/Streetview from junior/detect
!pip install streetlevel

/content/drive/MyDrive/黃俊翔_論文研究📚/Street_view/Streetview from junior/detect


In [ ]:
# prepare required functions
import math

def decode_polyline(polyline_str):
    '''Pass a Google Maps encoded polyline string; returns list of lat/lon pairs'''
    index, lat, lng = 0, 0, 0
    coordinates = []
    changes = {'latitude': 0, 'longitude': 0}

    # Coordinates have variable length when encoded, so just keep
    # track of whether we've hit the end of the string. In each
    # while loop iteration, a single coordinate is decoded.
    while index < len(polyline_str):
        # Gather lat/lon changes, store them in a dictionary to apply them later
        for unit in ['latitude', 'longitude']:
            shift, result = 0, 0
            while True:
                byte = ord(polyline_str[index]) - 63
                index += 1
                result |= (byte & 0x1f) << shift
                shift += 5
                if not byte >= 0x20:
                    break
            if (result & 1):
                changes[unit] = ~(result >> 1)
            else:
                changes[unit] = (result >> 1)
        lat += changes['latitude']
        lng += changes['longitude']
        coordinates.append((lat / 100000.0, lng / 100000.0))
    return coordinates

def getpathpoints(dirs,pts_distance=50):
  # polypath = dirs[0]['legs'][0]['steps'][0]['polyline']['points']
  polypath = dirs[0]['overview_polyline']['points']
  polypath_pts = decode_polyline(polypath)
  pts_coords = []
  for pt in range(1, len(polypath_pts)):
    lat1 = polypath_pts[pt-1][0]
    lng1 = polypath_pts[pt-1][1]
    lat2 = polypath_pts[pt][0]
    lng2 = polypath_pts[pt][1]
    azimuth = calculateBearing(lat1,lng1,lat2,lng2)
    coords = getInnerPoints(pts_distance,azimuth,lat1,lng1,lat2,lng2)
    if pt==1:
      pts_coords.append(coords)
    else:
      pts_coords.append(coords[1:])
  flatten_pts_coords = []
  for subl in pts_coords:
      for item in subl:
          flatten_pts_coords.append(item)
  return flatten_pts_coords

def getPathLength(lat1,lng1,lat2,lng2):
    '''calculates the distance between two lat, long coordinate pairs'''
    R = 6371000 # radius of earth in m
    lat1rads = math.radians(lat1)
    lat2rads = math.radians(lat2)
    deltaLat = math.radians((lat2-lat1))
    deltaLng = math.radians((lng2-lng1))
    a = math.sin(deltaLat/2) * math.sin(deltaLat/2) + math.cos(lat1rads) * math.cos(lat2rads) * math.sin(deltaLng/2) * math.sin(deltaLng/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d

def getDestinationLatLong(lat,lng,azimuth,distance):
    '''returns the lat an long of destination point
    given the start lat, long, aziuth, and distance'''
    R = 6378.1 #Radius of the Earth in km
    brng = math.radians(azimuth) #Bearing is degrees converted to radians.
    d = distance/1000 #Distance m converted to km
    lat1 = math.radians(lat) #Current dd lat point converted to radians
    lon1 = math.radians(lng) #Current dd long point converted to radians
    lat2 = math.asin(math.sin(lat1) * math.cos(d/R) + math.cos(lat1)* math.sin(d/R)* math.cos(brng))
    lon2 = lon1 + math.atan2(math.sin(brng) * math.sin(d/R)* math.cos(lat1), math.cos(d/R)- math.sin(lat1)* math.sin(lat2))
    #convert back to degrees
    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)
    return[lat2, lon2]

def calculateBearing(lat1,lng1,lat2,lng2):
    '''calculates the azimuth in degrees from start point to end point'''
    startLat = math.radians(lat1)
    startLong = math.radians(lng1)
    endLat = math.radians(lat2)
    endLong = math.radians(lng2)
    dLong = endLong - startLong
    dPhi = math.log(math.tan(endLat/2.0+math.pi/4.0)/math.tan(startLat/2.0+math.pi/4.0))
    if abs(dLong) > math.pi:
         if dLong > 0.0:
             dLong = -(2.0 * math.pi - dLong)
         else:
             dLong = (2.0 * math.pi + dLong)
    bearing = (math.degrees(math.atan2(dLong, dPhi)) + 360.0) % 360.0;
    return bearing

def getInnerPoints(interval,azimuth,lat1,lng1,lat2,lng2):
    '''returns every coordinate pair inbetween two coordinate
    pairs given the desired interval'''

    d = getPathLength(lat1,lng1,lat2,lng2)
    remainder, dist = math.modf((d / interval))
    counter = float(interval)
    coords = []
    coords.append([lat1,lng1])
    for distance in range(0,int(dist)):
        coord = getDestinationLatLong(lat1,lng1,azimuth,counter)
        counter = counter + float(interval)
        coords.append(coord)
    coords.append([lat2,lng2])
    return coords

def samepoint(line1, line2):
  result = False
  for x in line1:
    for y in line2:
      if x == y:
        result = True
        return x
        break
  return result

def find_intersection(p0, p1, p2, p3):
    # line1 = [p0.tolist(),p1.tolist()]
    # line2 = [p2.tolist(),p3.tolist()]
    line1 = [p0,p1]
    line2 = [p2,p3]
    samept = samepoint(line1,line2)
    if samept is not False:
      intersection_point = samept
    else:
      s10_x = p1[0] - p0[0]
      s10_y = p1[1] - p0[1]
      s32_x = p3[0] - p2[0]
      s32_y = p3[1] - p2[1]
      denom = s10_x * s32_y - s32_x * s10_y
      if denom == 0 : return math.nan # collinear
      denom_is_positive = denom > 0
      s02_x = p0[0] - p2[0]
      s02_y = p0[1] - p2[1]
      s_numer = s10_x * s02_y - s10_y * s02_x
      if (s_numer < 0) == denom_is_positive : return math.nan # no collision
      t_numer = s32_x * s02_y - s32_y * s02_x
      if (t_numer < 0) == denom_is_positive : return math.nan # no collision
      if (s_numer > denom) == denom_is_positive or (t_numer > denom) == denom_is_positive : return math.nan # no collision
      # collision detected
      t = t_numer / denom
      intersection_point = [ p0[0] + (t * s10_x), p0[1] + (t * s10_y) ]
    return intersection_point


In [ ]:
import geopandas
import pandas as pd
import plotly.graph_objects as go

# plot points on map
# pt1 = pd.DataFrame(Pts2downloadPano,columns=['lat','lon'])
# gdf = geopandas.GeoDataFrame(pt1, crs = {'init': 'epsg:4326'}, geometry=geopandas.points_from_xy(pt1.lon, pt1.lat)).to_crs(epsg=3857)
# gdf = geopandas.GeoDataFrame(pt1, geometry=geopandas.points_from_xy(pt1.lon, pt1.lat))

mapbox_access_token = "pk.eyJ1IjoiY2hpeWluZ2xpbiIsImEiOiJja24xNDB3eWkwaWh1Mm5vMnRud2cwdTZ1In0.mVM5aPE2TgbxeKo1il1CUQ"

## using plotly ##
def mapplotpoints(lat,lon,size=12,zoom=16,color='blue',name=None,text=None):
  # input gdf in ndarrays
  fig = go.Figure(go.Scattermapbox(
          lat= list(lat),
          lon= list(lon),
          mode='markers',
          name=name,
          marker=go.scattermapbox.Marker(
              size=size,
              color=color # named CSS color
          ),
          text=text
      ))
  fig.update_layout(
      autosize=True,
      hovermode='closest',
      mapbox=dict(
          accesstoken=mapbox_access_token,
          bearing=0,
          center=go.layout.mapbox.Center(
              # lat= np.mean(gdf['lat'].values),
              # lon= np.mean(gdf['lon'].values),
              lat= np.mean(lat),
              lon= np.mean(lon),
          ),
          pitch=0,
          zoom=zoom
      )
  )
  #fig.show()
  return fig


In [ ]:
import asyncio
from aiohttp import ClientSession
from streetlevel import streetview
import os
import math
from PIL import Image
import shutil

# Resize the downloaded image to 640x640 pixels
def resize_image(input_path, output_path, size):
    with Image.open(input_path) as img:
        img_resized = img.resize(size)
        img_resized.save(output_path)

# Asynchronous function to download and resize a street view image
async def download_and_resize_street_view(lat, lon, path1, path2, size):
    # Ensure the target folder exists
    if not os.path.exists(path1):
        os.makedirs(path1)
    if not os.path.exists(path2):
        os.makedirs(path2)

    async with ClientSession() as session:
        # Find the panorama
        pano = await streetview.find_panorama_async(lat, lon, session)
        # Define the file path for the original image
        original_image_path = os.path.join(path1, f"{pano.id}.jpg")
        # Check if the image already exists
        if os.path.exists(original_image_path):
            print(f"Image {pano.id}.jpg already exists. Using the existing image.")
        else:
            # Download the panorama image and save it
            await streetview.download_panorama_async(pano, original_image_path, session)
        # Resize the image after downloading
        resized_path = os.path.join(path2, f"{pano.id}resized.jpg")
        resize_image(original_image_path, resized_path, size)
        # Store the heading in degrees
        heading = pano.heading * 180 / math.pi

    return heading, resized_path

# Clean directory
def clean_directory(directory):
    if os.path.exists(directory):
        for file in os.listdir(directory):
            file_path = os.path.join(directory, file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
        print(f"Successfully cleaned up the directory: {directory}")
    else:
        print(f"The directory does not exist: {directory}")


In [ ]:
import requests
import matplotlib.image as mpimg
from matplotlib import gridspec
import io

#### Bounding boxes info extraction
def BBInfoExtraction(allbox,imgwidth,coor,dist_para,heading):
  # parameters for distance inferecing
  pole_Height = dist_para[0] # 40*304.8 set standard pole as 40 ft height above ground (mm)
  camera_fl = dist_para[1] # 1.5 camera focal length (mm)
  camera_sh = dist_para[2] # 4.55 camera sensor height (mm)
  searchdist = dist_para[3] # 10 distance coverage from the proposed pole distance (m)
  # extract pole info in a pano
  bearings = []
  LOB = []
  for bb in allbox:
    # bb = [x1, y1, x2, y2]
    dtheta = (bb[0]+bb[2])/2-imgwidth/2
    if dtheta > 0:
      bearing = dtheta/imgwidth*360
    else:
      bearing = 360 + dtheta/imgwidth*360
    bearing = (bearing + heading) %360
    bearings.append(bearing)

    pole_px = bb[3]-bb[1] # pixel height of detected pole bounding box
    if pole_px > 0:
      pole_dist = camera_fl*pole_Height*imageshape[0]/camera_sh/pole_px/1000 # calculated pole distance from camera (m)
      # compute LOB based on the proposed pole location +/- a search distance
      LOBstart = getDestinationLatLong(coor[0], coor[1], bearing, max(0,pole_dist-searchdist))
      LOBend = getDestinationLatLong(coor[0], coor[1], bearing, min(80,pole_dist+searchdist))
      LOB.append([LOBstart[0],LOBstart[1],LOBend[0],LOBend[1]])
    else:
      LOB.append([])
  return LOB, bearings

#### find intersections of predicted line of bearings (LOBs)
def LOBintersections(poleLOB, startpoint, nvg = 4, plotresult=False):
  def unnestlist(alist):
    unnestedlist = []
    for subl in alist:
      for item in subl:
        unnestedlist.append(item)
    return unnestedlist

  allintxn = []
  for gg in range(0,len(poleLOB)-nvg+1):
    subLOB = poleLOB[gg:gg+nvg] # subset of LOB group
    intxninsub = [] # all intersections in this subgroup
    for sg in range(0,nvg-1):
      vc = subLOB[sg] # current LOB group
      vo_temp = [x for i,x in enumerate(subLOB) if i!=sg] # all other LOB groups
      vo = unnestlist(vo_temp) # all other LOBs
      for i in range(0,len(vc)):
        for j in range(0,len(vo)):
          intersect = find_intersection(vc[i][0:2], vc[i][2:], vo[j][0:2], vo[j][2:])
          if not intersect is math.nan:
            intxninsub.append(intersect)
    # a[gg] = intxninsub
    allintxn.append(intxninsub)

  # extract unique intersections
  intxn_unnest = unnestlist(allintxn)
  intxn_unique = [i for n, i in enumerate(intxn_unnest) if i not in intxn_unnest[:n]]
  intxn_array = np.array(intxn_unique)

  ## first grouping: combine intersection generated by prediction error
  distMax = 1
  intxngroup = []
  for i in range(0,len(intxn_array)):
    if i not in unnestlist(intxngroup):
      pt0 = intxn_array[i]
      dist = [(lambda x: getPathLength(pt0[0],pt0[1],x[0], x[1]))(x) for x in intxn_array]
      closepts = [i for i, x in enumerate(np.array(dist) < distMax) if x]
      intxngroup.append(closepts)

  intxns = []
  for i in range(0,len(intxngroup)):
    lats = intxn_array[intxngroup[i],0]
    lons = intxn_array[intxngroup[i],1]
    intxns.append([np.mean(lats), np.mean(lons)])

  ## second grouping: combine intersections very close to each other
  distMax = 5
  intxngroup = []

  intxn_array2 = np.array(intxns)
  for i in range(0,len(intxn_array2)):
    if i not in unnestlist(intxngroup):
      pt0 = intxn_array2[i]
      dist = [(lambda x: getPathLength(pt0[0],pt0[1],x[0], x[1]))(x) for x in intxn_array2]
      closepts = [i for i, x in enumerate(np.array(dist) < distMax) if x]
      intxngroup.append(closepts)

  intxns2 = []
  for i in range(0,len(intxngroup)):
    lats = intxn_array2[intxngroup[i],0]
    lons = intxn_array2[intxngroup[i],1]
    intxns2.append([np.mean(lats), np.mean(lons)])

  ## check pole distance
  maxpoledist = 70 # maximum allowable distance b/t poles [urban: 125 ft (38 m); rural: 300 ft (91 m)]
  maxsegdist = 500 # maximum distance b/t route segments; output route points might be quite far away and belong to different road
  intxns_sorted = []
  intxns_index = []
  dist = [(lambda x: getPathLength(startpoint[0],startpoint[1],x[0],x[1]))(x) for x in intxns2]
  if dist:
    intxns_index.append(np.argmin(dist))
    intxns_sorted.append(intxns2[np.argmin(dist)])

  for i in range(1,len(intxns2)):
    prevpt = intxns_sorted[i-1]
    dist = np.array([[n, (lambda x: getPathLength(prevpt[0],prevpt[1],x[0],x[1]))(x)] for n, x in enumerate(intxns2) if n not in intxns_index])
    sortdist = dist[np.argsort(dist[:, 1])]
    intxns_index.append(sortdist[0,0])
    intxns_sorted.append(intxns2[int(sortdist[0,0])])

  intxns3 = intxns_sorted.copy()
  for i in range(1,len(intxns_sorted)-1):
    pt0 = intxns_sorted[i]
    pt1 = intxns_sorted[i+1]
    dist2next = getPathLength(pt0[0],pt0[1],pt1[0],pt1[1])
    if maxpoledist < dist2next < maxsegdist:
      n_pole_needed = int(dist2next // maxpoledist) # number of pole needed to insert
      dist2insertpole = dist2next/(n_pole_needed+1)
      bearing = calculateBearing(pt0[0],pt0[1],pt1[0],pt1[1])
      for j in range(0,n_pole_needed):
        intxns3.append(getDestinationLatLong(pt0[0],pt0[1],bearing,dist2insertpole*(j+1)))
  intxns_final = intxns3.copy()

  ## plot results
  if plotresult:
    finalintxns = intxns_final
    print('Number of intersections = {}'.format(len(finalintxns)))
    pointtext=[str(x) for x in PanoInfoFinal.index+1]

    fig = mapplotpoints(PanoInfoFinal['lat'],PanoInfoFinal['lon'],size = 8, name='street view location', color='DodgerBlue',text=pointtext)
    fig.add_trace(go.Scattermapbox(
            lat=np.array(finalintxns)[:,0],
            lon=np.array(finalintxns)[:,1],
            name='possible pole locations',
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=10,
                color='red',
                opacity=0.9
            )))

    for i in range(0,len(poleLOB)):
      lob = poleLOB[i]
      for j in range(0,len(lob)):
        fig.add_trace(
            go.Scattermapbox(
                lon = [np.array(poleLOB[i][j])[1], np.array(poleLOB[i][j])[3]],
                lat = [np.array(poleLOB[i][j])[0], np.array(poleLOB[i][j])[2]],
                mode = 'lines',
                line = dict(width = 1,color = 'Dodgerblue'),
                opacity=0.5
            )
        )
    fig.update_layout(
        height=650
    )
    return fig

  return intxns_final

In [ ]:
# read ROI route points (locations to download street view images)

workingdirectory = '/content/'

GalvestonRoutesPoints = pd.read_excel(workingdirectory + "galveston_route_point.xlsx",converters={'SEG_ID':str})
RouteIDs = GalvestonRoutesPoints['RTE_NM'].unique() # unique route ID
GalvestonRoutesPoints.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33321 entries, 0 to 33320
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RTE_NM  33321 non-null  object 
 1   SEG_ID  33321 non-null  object 
 2   Lon     33321 non-null  float64
 3   Lat     33321 non-null  float64
dtypes: float64(2), object(2)
memory usage: 1.0+ MB


In [ ]:
!pip install --upgrade numpy
%cd /content/drive/MyDrive/黃俊翔_論文研究📚/Street_view/Streetview from junior/detect/yolov5

import torch
import os

# PATCH torch.load to use weights_only=False
_original_torch_load = torch.load
def _patched_torch_load(f, *args, **kwargs):
    kwargs.setdefault('weights_only', False)
    return _original_torch_load(f, *args, **kwargs)
torch.load = _patched_torch_load

from detect import run

# 指定文件夹名称
folder_name = "/content/PolePredictionResults"

# 创建文件夹（如果文件夹已存在则不会报错）
os.makedirs(folder_name, exist_ok=True)

/content/drive/MyDrive/黃俊翔_論文研究📚/Street_view/Streetview from junior/detect/yolov5
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import numpy as np
import json
import time
import os
import pickle

dist_para = [40*304.8,1.5,4.55,10]
size = (640, 640)

# Specify the folder to save images
path1 = workingdirectory + 'StreetViewImages/'
path2 = workingdirectory + 'resized/'

savepreddir = workingdirectory + 'PolePredictionResults/'

# Checkpoint file to save progress (saved to Google Drive so it persists across sessions)
checkpoint_file = '/content/drive/MyDrive/黃俊翔_論文研究📚/Street_view/checkpoint_progress.pkl'

# Load checkpoint if exists
start_route = 0
totaldownload = 0
failed_downloads = 0
failed_requests = 0

if os.path.exists(checkpoint_file):
    try:
        with open(checkpoint_file, 'rb') as f:
            checkpoint = pickle.load(f)
            start_route = checkpoint.get('last_completed_route', 0) + 1
            totaldownload = checkpoint.get('totaldownload', 0)
            failed_downloads = checkpoint.get('failed_downloads', 0)
            failed_requests = checkpoint.get('failed_requests', 0)
        print(f"📍 Resuming from route {start_route + 1}/{len(RouteIDs)}")
        print(f"   Previous stats: downloads={totaldownload}, failed_downloads={failed_downloads}, failed_requests={failed_requests}")
    except Exception as e:
        print(f"⚠️  Warning: Could not load checkpoint: {e}")
        print("   Starting from beginning...")
        start_route = 0
else:
    print(f"🚀 Starting fresh processing of {len(RouteIDs)} routes")

clean_directory('/content/drive/MyDrive/論文研究📚/Street_view/Streetview from junior/detect/yolov5/runs/detect')

for rr in range(start_route, len(RouteIDs)):
  routeid = RouteIDs[rr]
  routepoints = GalvestonRoutesPoints.loc[GalvestonRoutesPoints['RTE_NM'] == routeid]
  Pts2downloadPano = routepoints[['Lat','Lon']].values.tolist()

  allpanoid = []
  for panoLatLon in Pts2downloadPano:
    try:
      pano = streetview.find_panorama(lat=panoLatLon[0], lon=panoLatLon[1])
      if bool(pano):
        panoid = pano.id
        panolat = pano.lat
        panolon = pano.lon
        allpanoid.append([panoid, panolat, panolon])
    except json.JSONDecodeError as e:
      failed_requests += 1
      print(f'\nJSONDecodeError at location ({panoLatLon[0]}, {panoLatLon[1]}): {str(e)}')
      print(f'Skipping this location and continuing... (Total failed requests: {failed_requests})')
      time.sleep(1)  # Add a small delay to avoid rate limiting
      continue
    except Exception as e:
      failed_requests += 1
      print(f'\nUnexpected error at location ({panoLatLon[0]}, {panoLatLon[1]}): {str(e)}')
      print(f'Skipping this location and continuing... (Total failed requests: {failed_requests})')
      time.sleep(1)
      continue
  PanoInfoFinal = pd.DataFrame(allpanoid,columns=['panoid','lat','lon']).drop_duplicates()

  intersections = []

  if len(PanoInfoFinal) > 3:
    poleview = []
    poleLOB = []

    for pp in PanoInfoFinal.index:
      print('\r','Progress: route = {}/{}; pano = {}/{}; download = {}; failed downloads = {}; failed requests = {}'.format(
          rr+1, len(RouteIDs), pp+1, len(PanoInfoFinal.index), totaldownload, failed_downloads, failed_requests), end = '')

      lat = PanoInfoFinal['lat'][pp]
      lon = PanoInfoFinal['lon'][pp]
      pano_coor = [PanoInfoFinal['lat'][pp], PanoInfoFinal['lon'][pp]]

      try:
        # Try to download and process the image
        heading, source = await download_and_resize_street_view(lat, lon, path1, path2, size)
        totaldownload = totaldownload + 1

        weights = '/content/drive/MyDrive/黃俊翔_論文研究📚/Street_view/Streetview from junior/detect/yolov5/runs/train/exp_J/best.pt'

        bounding_boxes = run(weights, source)
        image = Image.open(source)
        imageshape = np.shape(image)
        clean_directory(path2)

        # extract prediction and compute line of bearings
        LOB, bearings = BBInfoExtraction(bounding_boxes,imageshape[1],pano_coor,dist_para,heading)
        poleview.append([pano_coor, bearings])
        poleLOB.append(LOB)

      except Exception as e:
        # Skip corrupted/failed images and continue
        failed_downloads += 1
        print(f"\nWarning: Failed to process image at lat={lat}, lon={lon}. Error: {str(e)}")
        clean_directory(path2)  # Clean up any partial files
        continue

    startpoint = [PanoInfoFinal['lat'][0], PanoInfoFinal['lon'][0]]
    intersections = LOBintersections(poleLOB, startpoint)

  if intersections:
    poleLocations = pd.DataFrame(intersections,columns=['lat','lon'])
    poleLocations.to_excel(savepreddir + 'Pred_r{:0>4d}.xlsx'.format(rr+1), index = False)

  # Save checkpoint after each route (every 10 routes to reduce I/O)
  if (rr + 1) % 10 == 0 or rr == len(RouteIDs) - 1:
    try:
      checkpoint = {
          'last_completed_route': rr,
          'totaldownload': totaldownload,
          'failed_downloads': failed_downloads,
          'failed_requests': failed_requests,
          'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
      }
      with open(checkpoint_file, 'wb') as f:
          pickle.dump(checkpoint, f)
      print(f'\n💾 Checkpoint saved at route {rr+1}/{len(RouteIDs)}')
    except Exception as e:
      print(f'\n⚠️  Warning: Could not save checkpoint: {e}')

print(f"\n\n✅ Processing complete!")
print(f"Total successful downloads: {totaldownload}")
print(f"Total failed downloads: {failed_downloads}")
print(f"Total failed API requests: {failed_requests}")

# Delete checkpoint file when done
if os.path.exists(checkpoint_file):
    os.remove(checkpoint_file)
    print(f"🗑️  Checkpoint file deleted (processing complete)")

In [ ]:
# 🔧 Checkpoint管理工具 - 在運行主程序前使用
import os
import pickle

checkpoint_file = '/content/drive/MyDrive/黃俊翔_論文研究📚/Street_view/checkpoint_progress.pkl'

# 檢查當前進度
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'rb') as f:
        checkpoint = pickle.load(f)
    print("📊 當前進度:")
    print(f"   上次完成的 route: {checkpoint['last_completed_route'] + 1}/{len(RouteIDs)}")
    print(f"   成功下載數: {checkpoint['totaldownload']}")
    print(f"   下載失敗數: {checkpoint['failed_downloads']}")
    print(f"   API 請求失敗數: {checkpoint['failed_requests']}")
    print(f"   保存時間: {checkpoint['timestamp']}")
    print(f"\n   剩餘 routes: {len(RouteIDs) - checkpoint['last_completed_route'] - 1}")
    print(f"   完成百分比: {(checkpoint['last_completed_route'] + 1) / len(RouteIDs) * 100:.1f}%")
    print("\n⚠️  如果要從頭開始，請運行下面的重置代碼")
else:
    print("📍 沒有找到 checkpoint 文件")
    print("   程序將從頭開始運行")

# 如果要重置進度，取消下面這行的注釋並運行:
# if os.path.exists(checkpoint_file): os.remove(checkpoint_file); print("✅ Checkpoint 已刪除，將從頭開始")

In [ ]:
print(Pts2downloadPano)
print(PanoInfoFinal)

In [ ]:
# combine all predicted locations
# savepreddir = '/content/drive/MyDrive/黃俊翔_論文研究📚/Street_view/Streetview from junior/detect/PolePredictionResults/'
PredResultList = os.listdir(savepreddir)

allprediction = []

def unnestlist(alist):
  unnestedlist = []
  for subl in alist:
    for item in subl:
      unnestedlist.append(item)
  return unnestedlist

for i in range(0,len(PredResultList)):
# for i in range(0,20):
  pred_temp = pd.read_excel(savepreddir + PredResultList[i])
  allprediction.append(pred_temp.values.tolist())
  # if len(pred_temp) > 10:
  #   allprediction.append(pred_temp.values.tolist())


allprediction_T = pd.DataFrame(unnestlist(allprediction),columns=['Lat','Lon'])
allprediction_T.to_excel(workingdirectory + '/drive/MyDrive/黃俊翔_論文研究📚/Street_view/' + 'Galveston_test.xlsx', index = False)

allprediction_T

fig = mapplotpoints(allprediction_T['Lat'],allprediction_T['Lon'], size = 7, name='Predicted pole locations', color='orange', zoom = 15)
fig.update_layout(
    height=1200
)

In [ ]:
# import shutil
# import os

# # 定义源文件夹和目标路径
# source_folder = '/content/StreetViewImages'
# destination_path = '/content/drive/MyDrive/黃俊翔_論文研究📚/Street_view'

# # 确保目标文件夹存在，如果不存在则创建
# if not os.path.exists(destination_path):
#     os.makedirs(destination_path)

# # 移动文件夹
# shutil.move(source_folder, destination_path)

Error: Destination path '/content/drive/MyDrive/黃俊翔_論文研究📚/Street_view/StreetViewImages' already exists

In [ ]:
# compute accuracy
# PctAcc1: Percentage of the number of actual poles being within a buffer zone of a predicted pole (%)
# PctAcc2: Percentage of the number of predicted of poles being within a certain buffer zone of actual poles (%)

WHPoleLoc = pd.read_excel(workingdirectory + "PoleLocationGT.xlsx",sheet_name='pole') # actual pole locations
WHPolePred = pd.read_excel(workingdirectory + "AllPredictions.xlsx") # predicted pole locations

distMax = 2
polepredgroup = []

polepred = np.array(WHPolePred)
for i in range(0,len(polepred)):
  if i not in unnestlist(polepredgroup):
    pt0 = polepred[i]
    dist = [(lambda x: getPathLength(pt0[0],pt0[1],x[0], x[1]))(x) for x in polepred]
    closepts = [i for i, x in enumerate(np.array(dist) < distMax) if x]
    polepredgroup.append(closepts)

polepredgroup_final = []
for i in range(0,len(polepredgroup)):
  lats = polepred[polepredgroup[i],0]
  lons = polepred[polepredgroup[i],1]
  polepredgroup_final.append([np.mean(lats), np.mean(lons)])

bufferdist = [1,2,3,5,7,10,15] # buffer distance (meter)

PctAcc1 =[]
PctAcc2 =[]

for bb in range(0,len(bufferdist)):
  poles_in_Buffer1 = 0
  for i in range(0,len(polepredgroup_final)):
    pt0 = np.array(polepredgroup_final[i]) # current pole
    dist = [(lambda x: getPathLength(pt0[0],pt0[1],x[0], x[1]))(x) for x in WHPoleLoc.values.tolist()]
    if min(dist) < bufferdist[bb]:
      poles_in_Buffer1 += 1
  PctAcc1.append(poles_in_Buffer1/len(polepredgroup_final))

  poles_in_Buffer2 = 0
  for i in range(0,len(WHPoleLoc)):
    pt0 = np.array(WHPoleLoc.loc[i]) # current pole
    dist = [(lambda x: getPathLength(pt0[0],pt0[1],x[0], x[1]))(x) for x in polepredgroup_final]
    if min(dist) < bufferdist[bb]:
      poles_in_Buffer2 += 1
  PctAcc2.append(poles_in_Buffer2/len(WHPoleLoc))

# PRDresult = pd.DataFrame(np.array([np.array(bufferdist),np.array(PctAcc1)]).T,columns=['Buffer(m)','PDR'])
# PRDresult

PRDresult = pd.DataFrame(np.array([np.array(bufferdist),np.array(PctAcc1),np.array(PctAcc2)]).T,columns=['Buffer(m)','PDR1','PDR2'])
PRDresult

# **Save the predicted coordinate into excel file and export it to Google Drive.**

# **Compare with Ground truth dataset(For verifying)**

In [ ]:
fig = mapplotpoints(PanoInfoFinal['lat'][0:1],PanoInfoFinal['lon'][0:1],size = 1, name='Street view location', color='DodgerBlue')

# fig = mapplotpoints(np.array(WHPoleLoc)[:,0],np.array(WHPoleLoc)[:,1],size = 8, name='Actual pole locations', color='orange')

fig.add_trace(go.Scattermapbox(
        lat=np.array(WHPoleLoc)[:,0],
        lon=np.array(WHPoleLoc)[:,1],
        name='Actual pole locations',
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=15,
            color='orange',
            opacity=0.6
        )))

fig.add_trace(go.Scattermapbox(
        lat=np.array(polepredgroup_final)[:,0],
        lon=np.array(polepredgroup_final)[:,1],
        name='Predicted pole locations',
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='red',
            opacity=0.9
        )))


fig.update_layout(
    height=800
)